In [4]:
import os
import json
import torch

args_perm_id = 0
args_task_num = 5
args_class_num = 10
args_shot_num = 5
def collect_from_json(dataset, root, split):
    if split == "train":
        pth = os.path.join(
            root,
            dataset,
            "perm" + str(args_perm_id),
            f"{dataset}_{args_task_num}task_{args_class_num // args_task_num}way_{args_shot_num}shot.{split}.jsonl",
        )
    elif split in ["dev", "test"]:
        pth = os.path.join(root, dataset, f"{dataset}.{split}.jsonl")
    elif split == "stream":
        pth = os.path.join(
            root,
            dataset,
            f"stream_label_{args_task_num}task_{args_class_num // args_task_num}way.json",
        )
    else:
        raise ValueError(f'Split "{split}" value wrong!')
    if not os.path.exists(pth):
        raise FileNotFoundError(f"Path {pth} do not exist!")
    else:
        with open(pth) as f:
            if pth.endswith(".jsonl"):
                data = [json.loads(line) for line in f]
                if split == "train":
                    data = [list(i.values()) for i in data]
            else:
                data = json.load(f)
    # if split == "train":
    #     data = extract_single_dict(data)

    return data

In [8]:
dataset = 'ACE'
root = './data_incremental'
split = 'train'

In [24]:
data = [instance for t in collect_from_json(dataset, root, split)[1] for instance in t]

In [64]:
import torch


def get_one_hot_true_label_and_true_trigger(data_instance, num_label):
    true_label = []
    trigger_word = []
    seq_len = len(data_instance["piece_ids"]) + 1 # because start_index of piece_ids is 1 instead of 0
    for i in range(len(data_instance["label"])):
        if data_instance["label"][i] != 0:
            true_label.append(data_instance["label"][i])
            trigger_word.append(data_instance["span"][i])

    set_label_in_one_sentence = set(true_label)
    true_one_hot_trigger_vector = torch.zeros(num_label)
    for i in set_label_in_one_sentence:
        true_one_hot_trigger_vector += torch.eye(num_label)[i]

    true_one_hot_label_vector = torch.zeros(seq_len)
    trigger = []
    for i in trigger_word:
        trigger.extend(i)

    set_trig = set(trigger)
    for i in set_trig:
        true_one_hot_label_vector += torch.eye(seq_len)[i]
    return true_one_hot_trigger_vector, true_one_hot_label_vector

In [65]:
a, b = get_one_hot_true_label_and_true_trigger(data[0],10)

In [67]:
a

tensor([0., 1., 0., 0., 0., 0., 1., 0., 0., 0.])

In [68]:
b

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.])

In [52]:
print(len(data[0]['piece_ids']))
print(len(data[0]['label']))
print(len(data[0]['span']))

50
47
47


In [72]:
for instance in data:
    true_one_hot_trigger_vector, true_one_hot_label_vector = get_one_hot_true_label_and_true_trigger(instance, 10)
    instance['true_one_hot_trigger_vector'] = true_one_hot_trigger_vector.tolist()
    instance['true_one_hot_label_vector'] = true_one_hot_label_vector.tolist()

In [74]:
data[0]

{'piece_ids': [101,
  3960,
  1010,
  1045,
  2228,
  2008,
  1996,
  3114,
  7955,
  1999,
  1996,
  2148,
  1011,
  1011,
  2017,
  2113,
  1010,
  2034,
  1997,
  2035,
  1010,
  2057,
  2020,
  1011,
  1011,
  2043,
  5951,
  8573,
  2001,
  2700,
  2343,
  1010,
  2057,
  2018,
  2042,
  2542,
  2054,
  2057,
  2245,
  2001,
  2145,
  1037,
  11438,
  3842,
  2044,
  1996,
  2942,
  2162,
  1012,
  102],
 'label': [6,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'span': [[29, 29],
  [46, 47],
  [1, 1],
  [2, 2],
  [3, 3],
  [4, 4],
  [5, 5],
  [6, 6],
  [7, 7],
  [8, 8],
  [9, 9],
  [10, 10],
  [11, 11],
  [12, 12],
  [13, 13],
  [14, 14],
  [15, 15],
  [16, 16],
  [17, 17],
  [18, 18],
  [19, 19],
  [20, 20],
  [21, 21],
  [22, 22],
  [23, 23],
  [24, 24],
  [25, 25],
  [26, 26],
  [27, 27],
  [2

In [75]:
torch.cuda.is_available()

False

In [175]:
import torch


def get_one_hot_true_label_and_true_trigger(data_instance, num_label):
    true_label = []
    true_trigger = []
    seq_len = len(data_instance["piece_ids"]) # because start_index of piece_ids is 1 instead of 0
    
    for i in range(len(data_instance["label"])):
        if data_instance["label"][i] != 0:
            true_label.append(data_instance["label"][i])
            true_trigger.append(data_instance["span"][i])


    true_one_hot_label_vector = torch.zeros(num_label)
    true_one_hot_trigger_vector = torch.zeros(seq_len)

    set_label_in_one_sentence = set([label.item() for label in true_label])
    for i in set_label_in_one_sentence:
        true_one_hot_label_vector += torch.eye(num_label)[i]


    list_trigger = [trigger.tolist() for trigger in true_trigger]
    trigger = []
    for i in list_trigger:
        trigger.extend(i)

    set_trig_in_one_sentence = set(trigger)

    for i in set_trig_in_one_sentence:
        true_one_hot_trigger_vector += torch.eye(seq_len)[i]
    
    return true_one_hot_trigger_vector, true_one_hot_label_vector

In [39]:
device = 'cpu'
def get_one_hot_true_label_and_true_trigger(data_instance, num_label):
    true_label = []
    true_trigger = []
    seq_len = len(data_instance["piece_ids"]) # because start_index of piece_ids is 1 instead of 0
    matrix_word_is_label = torch.zeros(seq_len, num_label,dtype=int)
    for i in range(len(data_instance["label"])):
        if data_instance["label"][i] != 0:
            true_label.append(data_instance["label"][i])
            true_trigger.append(data_instance["span"][i])
            for word_is_trigger in data_instance['span'][i]:
                matrix_word_is_label[word_is_trigger,data_instance['label'][i]] = 1


    true_one_hot_label_vector = torch.zeros(num_label)
    true_one_hot_trigger_vector = torch.zeros(seq_len)

    set_label_in_one_sentence = set([label.item() for label in true_label])
    for i in set_label_in_one_sentence:
        true_one_hot_label_vector += torch.eye(num_label)[i]


    list_trigger = [trigger.tolist() for trigger in true_trigger]
    trigger = []
    for i in list_trigger:
        trigger.extend(i)

    set_trig_in_one_sentence = set(trigger)

    for i in set_trig_in_one_sentence:
        true_one_hot_trigger_vector += torch.eye(seq_len)[i]
    true_one_hot_trigger_vector = true_one_hot_trigger_vector.to(device)
    true_one_hot_label_vector = true_one_hot_label_vector.to(device)
    matrix_word_is_label = matrix_word_is_label.to(device)
    return true_one_hot_trigger_vector, true_one_hot_label_vector, matrix_word_is_label

def true_label_and_trigger(train_x,train_y,train_masks, train_span, class_num):
    num_instance = len(train_x)
    true_one_hot_label_vectors = []
    true_one_hot_trigger_vectors = []
    golden_matrix = []
    for i in range(num_instance):
        data_instace={
            'piece_ids': train_x[i],
            'label': train_y[i],
            'span': train_span[i],
            'mask': train_masks[i]
        }

        true_one_hot_trigger_vector, true_one_hot_label_vector, matrix_word_is_label= get_one_hot_true_label_and_true_trigger(data_instance=data_instace,num_label=class_num)
        true_one_hot_trigger_vectors.append(true_one_hot_trigger_vector)
        true_one_hot_label_vectors.append(true_one_hot_label_vector)
        golden_matrix.append(matrix_word_is_label)
    true_one_hot_trigger_vectors = torch.stack([x.to(device) for x in true_one_hot_trigger_vectors])
    true_one_hot_label_vectors = torch.stack([x.to(device) for x in true_one_hot_label_vectors])
    pi_golden_matrix = torch.stack([x.to(device) for x in golden_matrix])
    return true_one_hot_trigger_vectors, true_one_hot_label_vectors, pi_golden_matrix



In [40]:
import torch
train_x = torch.tensor([[  101,  6398,  1024,  6175,  2003,  2025,  1996,  2069,  2510,  2564,
          2040,  2363,  1037,  6302,  1998,  3661,  1012,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0],
        [  101, 21524,  1998,  2037,  2079, 24968,  5611,  6956, 19974,  2224,
          1996,  2773,  1036,  1036,  6139,  1005,  1005,  2043,  9694,  2008,
          3956,  2681,  1996,  2225,  2924,  1998, 14474,  1998,  4487, 11512,
          9286,  3644,  7617,  1012,   102,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0],
        [  101,  1045,  2079,  1050,  1005,  1056,  2228,  2008,  1005,  1055,
          3243,  8321,  2004,  2000,  2054,  2002,  2626,  2033,  2055,  1012,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0],
        [  101,  2358, 23111,  6582,  1010,  2040,  2001,  2809,  2706,  6875,
          1010,  2018,  3041,  2042,  3331,  2007,  2014,  2388,  2006,  1996,
          3042,  1010,  1998,  5112,  2039,  3038,  1037,  2450,  2016,  2018,
         11834,  3064,  2007,  3784,  2018,  2074,  3369,  2012,  2014,  2341,
          1010,  4614,  2056,  1012,   102,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0]])

train_y = [torch.tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), torch.tensor([2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0]), torch.tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), torch.tensor([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])]
train_masks = torch.tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]])
train_span = [torch.tensor([[15, 15],
        [ 1,  1],
        [ 2,  2],
        [ 3,  3],
        [ 4,  4],
        [ 5,  5],
        [ 6,  6],
        [ 7,  7],
        [ 8,  8],
        [ 9,  9],
        [10, 10],
        [11, 11],
        [12, 12],
        [13, 13],
        [14, 14],
        [16, 16]]), torch.tensor([[21, 21],
        [ 1,  1],
        [ 2,  2],
        [ 3,  3],
        [ 4,  4],
        [ 5,  5],
        [ 6,  6],
        [ 7,  7],
        [ 8,  8],
        [ 9,  9],
        [10, 10],
        [11, 11],
        [12, 12],
        [13, 13],
        [14, 14],
        [15, 15],
        [16, 16],
        [17, 17],
        [18, 18],
        [19, 19],
        [20, 20],
        [22, 22],
        [23, 23],
        [24, 24],
        [25, 25],
        [26, 26],
        [27, 27],
        [28, 28],
        [29, 29],
        [30, 30],
        [31, 31],
        [32, 32],
        [33, 33]]), torch.tensor([[16, 16],
        [ 1,  1],
        [ 2,  2],
        [ 3,  3],
        [ 4,  4],
        [ 5,  5],
        [ 6,  6],
        [ 7,  7],
        [ 8,  8],
        [ 9,  9],
        [10, 10],
        [11, 11],
        [12, 12],
        [13, 13],
        [14, 14],
        [15, 15],
        [17, 17],
        [18, 18],
        [19, 19]]), torch.tensor([[20, 20],
        [30, 31],
        [ 1,  1],
        [ 2,  2],
        [ 3,  3],
        [ 4,  4],
        [ 5,  5],
        [ 6,  6],
        [ 7,  7],
        [ 8,  8],
        [ 9,  9],
        [10, 10],
        [11, 11],
        [12, 12],
        [13, 13],
        [14, 14],
        [15, 15],
        [16, 16],
        [17, 17],
        [18, 18],
        [19, 19],
        [21, 21],
        [22, 22],
        [23, 23],
        [24, 24],
        [25, 25],
        [26, 26],
        [27, 27],
        [28, 28],
        [29, 29],
        [32, 32],
        [33, 33],
        [34, 34],
        [35, 35],
        [36, 36],
        [37, 37],
        [38, 38],
        [39, 39],
        [40, 40],
        [41, 41],
        [42, 42],
        [43, 43]])]

In [41]:
true_trigger, true_label, pi_golden = true_label_and_trigger(train_x=train_x,train_y=train_y,train_masks=train_masks,train_span=train_span,class_num=10)

In [42]:
true_label[0]

tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.])

In [43]:
train_y[0]

tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [50]:
pi_golden[3][30]

tensor([0, 1, 0, 0, 0, 0, 0, 0, 0, 0])

In [51]:
import numpy as np

# Giả sử bạn có các ma trận sau:
# pi_star: Ma trận alignment (sau khi tính toán thông qua OT)
pi_star = np.array([[0.3, 0.7, 0.1],
                    [0.5, 0.4, 0.1],
                    [0.2, 0.6, 0.3]])

# pi_g: Ma trận golden (true trigger labels)
pi_g = np.array([[0, 1, 0],  # w1 có nhãn đúng là t2
                 [1, 0, 0],  # w2 có nhãn đúng là t1
                 [0, 0, 1]]) # w3 có nhãn đúng là t3

# Nhân ma trận pi_star với pi_g (theo từng phần tử)
pi_star_weighted = pi_star * pi_g

# Tính tổng theo chiều ngang (axis=1)
pi_star_sum = pi_star_weighted.sum(axis=1)

print("Pi_star weighted by golden matrix (sum along axis 1):")
print(pi_star_sum)


Pi_star weighted by golden matrix (sum along axis 1):
[0.7 0.5 0.3]


In [65]:
pi_star_1 = torch.tensor([
    [[0.3, 0.7, 0.1,-1.0], [0.5, 0.4, 0.1,-1.0], [0.2, 0.6, 0.3,-1.0]],
    [[0.5, 0.4, 0.1,-2.0], [0.3, 0.7, 0.1,-2.0], [0.2, 0.6, 0.3,-2.0]]]
)
pi_star_2 = torch.tensor([
    [[0.3, 0.7, 0.1,-1.0], [0.5, 0.4, 0.1,-1.0], [0.2, 0.6, 0.3,-1.0]],
    [[0.5, 0.4, 0.1,-2.0], [0.3, 0.7, 0.1,-2.0], [0.2, 0.6, 0.3,-2.0]]]
)


In [113]:
import torch


def L_task(pi_star, y_true):
    """
    Tính Loss Task (Negative Log-Likelihood Loss) cho mỗi batch dữ liệu.

    Arguments:
    - pi_star (Tensor): Tensor có kích thước (batch_size, seq_len), chứa xác suất dự đoán cho từng từ và nhãn.
    - y_true (Tensor): Tensor có kích thước (batch_size, seq_len), chứa nhãn thực tế (labels) cho từng từ.

    Return:
    - loss (Tensor): giá trị loss trung bình cho cả batch.
    """

    # Chỉ tính log của pi_star mà không có phần (1 - pi_star)
    loss = -torch.log((torch.sum(pi_star*y_true,dim=-1))).mean()

    return loss


# Ví dụ sử dụng:
batch_size = 2  # Số câu trong batch
seq_len = 4  # Số từ trong mỗi câu

# Giả sử chúng ta có xác suất pi_star và nhãn thực tế y_true cho mỗi câu trong batch
pi_star = torch.tensor(
    [
        [[0.3, 0.7, 0.1, -1.0], [0.5, 0.4, 0.1, -1.0], [0.2, 0.6, 0.3, -1.0]],
        [[0.5, 0.4, 0.1, -2.0], [0.3, 0.7, 0.1, -2.0], [0.2, 0.6, 0.3, -2.0]],
    ]
)

y_true = torch.tensor(
    [
        [[0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0]],
        [[1, 0, 0, 0], [0, 1, 0, 0], [0, 1, 0, 0]],
    ]
)

# Tính L_task
loss_task = L_task(pi_star, y_true)
print(loss_task)

tensor(0.6357)


In [117]:
log_probs = torch.sum(pi_star * y_true, dim=-1)
log_probs = torch.log(log_probs + 1e-10)
loss = -log_probs.mean()
loss

tensor(0.6357)

In [110]:
-torch.log((torch.sum(pi_star*y_true,dim=-1))).mean()

tensor(0.6357)

In [94]:
torch.log(pi_star)*y_true

tensor([[[-0.0000, -0.3567, -0.0000,     nan],
         [-0.6931, -0.0000, -0.0000,     nan],
         [-0.0000, -0.0000, -1.2040,     nan]],

        [[-0.6931, -0.0000, -0.0000,     nan],
         [-0.0000, -0.3567, -0.0000,     nan],
         [-0.0000, -0.5108, -0.0000,     nan]]])

In [78]:
mul = -torch.log(pi_star_1*pi_star_2)

In [80]:
mul

tensor([[[ 2.4079,  0.7133,  4.6052, -0.0000],
         [ 1.3863,  1.8326,  4.6052, -0.0000],
         [ 3.2189,  1.0217,  2.4079, -0.0000]],

        [[ 1.3863,  1.8326,  4.6052, -1.3863],
         [ 2.4079,  0.7133,  4.6052, -1.3863],
         [ 3.2189,  1.0217,  2.4079, -1.3863]]])

In [82]:
mul.sum(dim=[1,2])/(mul.size(1)*mul.size(2))

tensor([1.8499, 1.5033])

In [64]:
pi_star.sum(dim=-1).size()

torch.Size([2, 3])

In [24]:
data_instance = {
    "piece_ids": train_x[3],
    "label": train_y[3],
    "span": train_span[3],
    "mask": train_masks[3],
}
# a, b = get_one_hot_true_label_and_true_trigger(data_instance=data_instance,num_label=10)

In [32]:
matrix_word_is_label = torch.zeros(len(data_instance['piece_ids']),10)
for i in range(len(data_instance["label"])):
    if data_instance["label"][i] != 0:
        # print(data_instance["label"][i])

    #     true_label.append(data_instance["label"][i])
    #     true_trigger.append(data_instance["span"][i])
        for word_is_trigger in data_instance['span'][i]:
            # print(word_is_trigger)
            matrix_word_is_label[word_is_trigger,data_instance['label'][i]] = 1


In [38]:
matrix_word_is_label[20:33]

tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [183]:
b

tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.])

In [197]:
for i in range(len(train_span)):
    print(train_span[i].size(-1))


2
2
2
2


In [126]:
import torch
test = [torch.tensor([[1,2,3],[4,5,6]]),torch.tensor([[7,8,9],[10,11,12]])]


In [145]:
import torch

# Giả sử bạn có các tensor sau:
# `last_hidden_state` có kích thước [batch_size, seqlen, hidden_dim]
last_hidden_state = torch.randn(4, 122, 768)  # Kích thước giả định

# `mask` có kích thước [batch_size, seqlen], giá trị 1 cho token thực sự và 0 cho padding
mask = torch.randint(0, 2, (4, 122))  # Ví dụ: 1 cho token thực sự, 0 cho padding



In [151]:
def get_true_hidden_state_without_padding(last_hidden_state, masks):
    masks = masks.unsqueeze(-1)
    mask_hidden_state = last_hidden_state * masks
    true_hidden_state_without_padding = mask_hidden_state.view(-1, 768)[
        masks.view(-1) == 1
    ]
    return true_hidden_state_without_padding  # [sum_true_token, hidden_dim]

In [152]:
true_embedding = get_true_hidden_state_without_padding(last_hidden_state,mask)

In [156]:
masked_hidden_state = last_hidden_state * mask.unsqueeze(-1)
masked_hidden_state.size()
masked_hidden_state = masked_hidden_state.view(-1, 768)[mask.view(-1) == 1]

In [157]:
masked_hidden_state.size()

torch.Size([249, 768])

In [153]:
true_embedding.size()

torch.Size([249, 768])

In [158]:
# Giả sử bạn có các tensor như sau:
batch_size = 4
seq_len = 6

# Các xác suất trigger cho từng token trong câu (p_wi)
p_wi = torch.sigmoid(torch.randn(batch_size, seq_len))  # [4, 6]

# Các nhãn thực tế (true_trig)
true_trig = torch.randint(0, 2, (batch_size, seq_len))  # [4, 6] với giá trị 0 hoặc 1

# Attention mask (masks), 1 cho token thực, 0 cho token padding
masks = torch.tensor([[1, 1, 1, 1, 0, 0],  # Câu 1 có 4 token thực
                      [1, 1, 1, 0, 0, 0],  # Câu 2 có 3 token thực
                      [1, 1, 1, 1, 1, 0],  # Câu 3 có 5 token thực
                      [1, 1, 1, 1, 1, 1]]) # Câu 4 có 6 token thực (không có padding)

# Tính loss TI
# loss = compute_loss_TI(p_wi, true_trig, masks)
# print("Loss TI:", loss.item())


In [161]:
p_wi[1][masks[1] == 1]

tensor([0.8904, 0.4555, 0.2070])

In [162]:
batch_size = 4
seq_len = 5
num_classes = 3

# Tạo dữ liệu giả cho pi_star và pi_golden (giả sử xác suất phân bố đều cho pi_star, và pi_golden là các vector nhãn)
pi_star = torch.rand(batch_size, seq_len, num_classes)  # Xác suất dự đoán ngẫu nhiên từ [0, 1]
pi_star = pi_star / pi_star.sum(dim=-1, keepdim=True)  # Chuẩn hóa về tổng = 1 (xác suất)

pi_golden = torch.randint(0, 2, (batch_size, seq_len, num_classes)).float()  # Nhãn thực (0 hoặc 1)


In [169]:
torch.sum(pi_golden*pi_star,dim=-1)

tensor([[0.1151, 0.4943, 1.0000, 0.3761, 0.3852],
        [0.4031, 0.7366, 1.0000, 0.8397, 0.1146],
        [1.0000, 0.6542, 0.8769, 1.0000, 0.4036],
        [0.8367, 0.6056, 0.7390, 0.0000, 0.3532]])

In [167]:
pi_golden*pi_star

tensor([[[0.0000, 0.1151, 0.0000],
         [0.1963, 0.2979, 0.0000],
         [0.2573, 0.4445, 0.2983],
         [0.0000, 0.3761, 0.0000],
         [0.0000, 0.3852, 0.0000]],

        [[0.4031, 0.0000, 0.0000],
         [0.0000, 0.3005, 0.4361],
         [0.4377, 0.2548, 0.3074],
         [0.3200, 0.0000, 0.5196],
         [0.0246, 0.0000, 0.0900]],

        [[0.2333, 0.3558, 0.4110],
         [0.1819, 0.0000, 0.4723],
         [0.6229, 0.0000, 0.2540],
         [0.2883, 0.2216, 0.4901],
         [0.0000, 0.0000, 0.4036]],

        [[0.0000, 0.3779, 0.4588],
         [0.0000, 0.6056, 0.0000],
         [0.2351, 0.5038, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.3532, 0.0000, 0.0000]]])

In [208]:
def sinkhorn_pytorch(M, a, b, lambda_sh, numItermax=1000, stopThr=5e-3):
    u = torch.ones_like(a) / a.size(0)
    v = torch.zeros_like(b)
    K = torch.exp(-M * lambda_sh)

    cpt = 0
    err = 1.0

    def condition(cpt, u, v, err):
        return cpt < numItermax and err > stopThr

    def v_update(u, v):
        v = b / torch.matmul(K.t(), u)
        u = a / torch.matmul(K, v)
        return u, v

    def no_v_update(u, v):
        return u, v

    def err_f1(K, u, v, b):
        bb = v * torch.matmul(K.t(), u)
        err = torch.norm(torch.sum(torch.abs(bb - b), dim=0), p=float('inf'))
        return err

    def err_f2(err):
        return err

    def loop_func(cpt, u, v, err):
        u = a / torch.matmul(K, b / torch.matmul(u.T, K).T)
        cpt = cpt + 1
        if cpt % 20 == 1 or cpt == numItermax:
            u, v = v_update(u, v)
            err = err_f1(K, u, v, b)
        else:
            u, v = no_v_update(u, v)
            err = err_f2(err)
        return cpt, u, v, err

    while condition(cpt, u, v, err):
        cpt, u, v, err = loop_func(cpt, u, v, err)

    sinkhorn_divergences = torch.sum(u * torch.matmul(K * M, v), dim=0)
    return sinkhorn_divergences

def compute_pi_star(M, a, b, lambda_sh, numItermax=1000, stopThr=5e-3):
    # Gọi hàm Sinkhorn để tính u, v và ma trận K
    u, v, K = sinkhorn_pytorch(M, a, b, lambda_sh, numItermax, stopThr)
    
    # Tính ma trận đồng nhất tối ưu pi_star bằng công thức pi_star = diag(u) * K * diag(v)
    pi_star = torch.diag(u) @ K @ torch.diag(v)
    
    return pi_star

# batch_size = 4
# num_class = 11
# num_token = 122

# word_preference = torch.randn(batch_size,num_token)
# type_preference = torch.randn(batch_size,num_class)
# cost_matrix = torch.randn(batch_size,num_class,num_token)


In [209]:
num_class, num_token = 11, 122

# Tạo ma trận chi phí ngẫu nhiên kích thước (n, m) = (11, 122)
M = torch.rand(num_class, num_token)

# Tạo các vector a và b là phân phối xác suất (tổng = 1)
a = torch.tensor([1.0 / num_class] * num_class)  # Phân phối xác suất cho a (có tổng = 1)
b = torch.tensor([1.0 / num_token] * num_token)  # Phân phối xác suất cho b (có tổng = 1)

# Tham số điều chỉnh regularization
lambda_sh = 20

In [ ]:
x = torch.randn([4,122,768])
y = torch.randn([11,768])
(x.unsqueeze(2)-y.unsqueeze(0).unsqueeze(0)).size()

torch.Size([4, 122, 11, 768])

In [226]:
x1=x.unsqueeze(2).repeat(1,1,11,1)
y1=y.unsqueeze(0).unsqueeze(0).repeat(4,122,1,1)

In [234]:
(1-torch.nn.functional.cosine_similarity(x1=x1,x2=y1,dim=3)).size()

torch.Size([4, 122, 11])

In [237]:
import torch
test = torch.randint(1,20,(4,11,2))

In [247]:
list(dict.fromkeys(test[0].flatten().tolist()))

[2, 8, 1, 12, 7, 5, 17, 16, 11, 14, 13, 15, 6]

In [248]:
test[0].flatten().tolist()

[2, 8, 1, 12, 7, 1, 5, 17, 16, 11, 14, 1, 13, 5, 11, 5, 15, 2, 17, 6, 11, 1]

In [252]:
a = [2, 2, 8, 8, 1, 1, 12, 13, 7, 5, 17, 16, 11,2, 14, 13, 15, 6]
a

[2, 2, 8, 8, 1, 1, 12, 13, 7, 5, 17, 16, 11, 2, 14, 13, 15, 6]

In [ ]:
list(dict.fromkeys(a))

[2, 8, 1, 12, 13, 7, 5, 17, 16, 11, 14, 15, 6]

In [255]:
x = torch.rand([4,122,768])

In [260]:
x[0][list(dict.fromkeys(a)),:].size()

torch.Size([13, 768])

In [261]:
linear = torch.nn.Linear(768,1)
input = torch.rand([17,768])

In [271]:
output = torch.sigmoid(linear(input))
output1 = (linear(input))

In [275]:
output1.squeeze(1)

tensor([-0.3153, -0.2226, -0.1029, -0.2959, -0.4649, -0.2998, -0.5438, -0.1999,
        -0.1981, -0.6199, -0.4724, -0.2184, -0.0811, -0.1304, -0.4779, -0.1843,
        -0.3864], grad_fn=<SqueezeBackward1>)

In [277]:
output1.size()

torch.Size([17, 1])

In [269]:
torch.softmax(output.squeeze(1),dim=0)

tensor([0.0576, 0.0632, 0.0713, 0.0588, 0.0496, 0.0585, 0.0458, 0.0647, 0.0648,
        0.0425, 0.0492, 0.0635, 0.0728, 0.0693, 0.0490, 0.0657, 0.0537],
       grad_fn=<SoftmaxBackward0>)

In [286]:
def get_true_y(y):
    true_y = []
    for i in range(len(y)):
        filter_y = (y[i] != 0).int()
        true_y.append(filter_y)
    return true_y

test = torch.randint(0,10,[4,10])

In [288]:
test

tensor([[6, 9, 6, 3, 6, 7, 5, 4, 6, 6],
        [6, 4, 7, 1, 3, 0, 1, 7, 9, 3],
        [0, 0, 2, 1, 9, 3, 3, 9, 4, 7],
        [9, 1, 3, 8, 7, 9, 7, 3, 6, 7]])

In [287]:
get_true_y(test)

[tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.int32),
 tensor([1, 1, 1, 1, 1, 0, 1, 1, 1, 1], dtype=torch.int32),
 tensor([0, 0, 1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.int32),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.int32)]

In [294]:


def compute_loss_TI(p_wi, true_y):
    loss_TI = 0.0
    for i in range(len(true_y)):
        # mask padding token
        loss_TI += -torch.dot(true_y[i].float(), torch.log(p_wi[i])) - torch.dot(
            (1 - true_y[i].float()), torch.log(1 - p_wi[i])
        )

    return loss_TI / len(true_y)


p_wi = [torch.tensor([0.9,0.05,0.1]), torch.tensor([0.6,0.1,0.1,0.2])]
true_y = [torch.tensor([1,0,0]),torch.tensor([1,0,0,0])]
compute_loss_TI(p_wi,true_y)

tensor(0.6034)

In [328]:
def get_true_y(y,num_class):
    true_trig,true_label = [],[]
    for i in range(len(y)):
        true_label_loop = torch.zeros(num_class)
        set_label = set(y[i].tolist())
        for label in set_label:
            if label != 0:
                true_label_loop += torch.nn.functional.one_hot(torch.tensor(label),num_classes=num_class)
        
        filter_y = (y[i] != 0).int()
        true_trig.append(filter_y)
        true_label.append(true_label_loop)
    return true_trig,true_label

In [329]:
y = [torch.tensor([2,3,5,0,0,0]),torch.tensor([7,8,0]),torch.tensor([1,1,2,0])]
num_class = 10

In [331]:
true_trig, true_label =get_true_y(y,num_class)

In [332]:
true_trig

[tensor([1, 1, 1, 0, 0, 0], dtype=torch.int32),
 tensor([1, 1, 0], dtype=torch.int32),
 tensor([1, 1, 1, 0], dtype=torch.int32)]

In [333]:
true_label

[tensor([0., 0., 1., 1., 0., 1., 0., 0., 0., 0.]),
 tensor([0., 0., 0., 0., 0., 0., 0., 1., 1., 0.]),
 tensor([0., 1., 1., 0., 0., 0., 0., 0., 0., 0.])]

In [318]:


def get_one_hot_true_label_and_true_trigger(data_instance, num_label):
    true_label = []
    true_trigger = []
    seq_len = len(
        data_instance["piece_ids"]
    )  # because start_index of piece_ids is 1 instead of 0
    matrix_word_is_label = torch.zeros(seq_len, num_label, dtype=int)
    for i in range(len(data_instance["label"])):
        if data_instance["label"][i] != 0:
            true_label.append(data_instance["label"][i])
            true_trigger.append(data_instance["span"][i])
            for word_is_trigger in data_instance["span"][i]:
                matrix_word_is_label[word_is_trigger, data_instance["label"][i]] = 1

    true_one_hot_label_vector = torch.zeros(num_label)
    true_one_hot_trigger_vector = torch.zeros(seq_len)

    set_label_in_one_sentence = set([label.item() for label in true_label])
    for i in set_label_in_one_sentence:
        true_one_hot_label_vector += torch.eye(num_label)[i]

    list_trigger = [trigger.tolist() for trigger in true_trigger]
    trigger = []
    for i in list_trigger:
        trigger.extend(i)

    set_trig_in_one_sentence = set(trigger)

    for i in set_trig_in_one_sentence:
        true_one_hot_trigger_vector += torch.eye(seq_len)[i]
    true_one_hot_trigger_vector = true_one_hot_trigger_vector.to(device)
    true_one_hot_label_vector = true_one_hot_label_vector.to(device)
    return true_one_hot_trigger_vector, true_one_hot_label_vector, matrix_word_is_label


def true_label_and_trigger(train_x, train_y, train_masks, train_span, class_num):
    num_instance = len(train_x)
    true_one_hot_label_vectors = []
    true_one_hot_trigger_vectors = []
    golden_matrix = []
    for i in range(num_instance):
        data_instace = {
            "piece_ids": train_x[i],
            "label": train_y[i],
            "span": train_span[i],
            "mask": train_masks[i],
        }

        true_one_hot_trigger_vector, true_one_hot_label_vector, matrix_word_is_label = (
            get_one_hot_true_label_and_true_trigger(
                data_instance=data_instace, num_label=class_num
            )
        )
        true_one_hot_trigger_vectors.append(true_one_hot_trigger_vector)
        true_one_hot_label_vectors.append(true_one_hot_label_vector)
        golden_matrix.append(matrix_word_is_label)
    true_one_hot_trigger_vectors = torch.stack(
        [x.to(device) for x in true_one_hot_trigger_vectors]
    )
    true_one_hot_label_vectors = torch.stack(
        [x.to(device) for x in true_one_hot_label_vectors]
    )
    pi_golden_matrix = torch.stack([x.to(device) for x in golden_matrix])
    return true_one_hot_trigger_vectors, true_one_hot_label_vectors, pi_golden_matrix



In [334]:
last_hidden_state_order = [torch.rand([11,768]),torch.rand([5,768]),torch.rand([6,768]),torch.rand(20,768)]

In [338]:
label_embedding = torch.rand([11,768])

In [341]:
label_embedding.unsqueeze(0).repeat([5,1,1]).size()

torch.Size([5, 11, 768])

In [345]:
last_hidden_state_order[1].unsqueeze(1).repeat([1,11,1]).size()

torch.Size([5, 11, 768])

In [348]:
(1-torch.nn.functional.cosine_similarity(label_embedding.unsqueeze(0).repeat([5,1,1]),last_hidden_state_order[1].unsqueeze(1).repeat([1,11,1]),dim=-1)).size()

torch.Size([5, 11])

In [350]:
def compute_cost_transport(last_hidden_state_order, label_embedding, num_classes = 11):
    # last_hidden_state_order: [batch_size, num_span, hidden_dim]
    # label_embedding: [num_class, hidden_dim]
    
    batch_size = len(last_hidden_state_order)
    cost_matrix = []
    for i in range(batch_size):
        num_span = last_hidden_state_order[i].size(0)
        label_embedding_scale = label_embedding.unsqueeze(0).repeat([num_span,1,1])
        last_hidden_state_order_scale = last_hidden_state_order[i].unsqueeze(1).repeat([1,num_classes,1])
        cost = 1-torch.nn.functional.cosine_similarity(last_hidden_state_order_scale,label_embedding_scale,dim=-1)
        cost_matrix.append(cost)
    return cost_matrix

In [351]:
cost_matrix = compute_cost_transport(last_hidden_state_order=last_hidden_state_order,label_embedding=label_embedding)

In [354]:
for i in range(len(cost_matrix)):
    print(f'size of cost_matrix[{i}]: {cost_matrix[i].size()}')

size of cost_matrix[0]: torch.Size([11, 11])
size of cost_matrix[1]: torch.Size([5, 11])
size of cost_matrix[2]: torch.Size([6, 11])
size of cost_matrix[3]: torch.Size([20, 11])


In [389]:
linear = torch.nn.Linear(768,1)
D_W_P = []
for sentence in range(len(last_hidden_state_order)):
    D_W_P.append(torch.softmax(torch.sigmoid(linear(last_hidden_state_order[sentence])).flatten(),dim=-1))

In [390]:
torch.sum(D_W_P[0])

tensor(1., grad_fn=<SumBackward0>)

In [364]:
e_cls = torch.rand([4,768])
e_cls_scale = e_cls.unsqueeze(1).repeat([1,11,1])

In [365]:
e_cls_scale.size()

torch.Size([4, 11, 768])

In [367]:
label_embedding_scale = label_embedding.unsqueeze(0).repeat([4,1,1])

In [369]:
concat = torch.cat([e_cls_scale,label_embedding_scale],dim=-1)

In [371]:
ffp = torch.nn.Linear(768*2,1)

In [398]:
D_T_P = torch.softmax(torch.sigmoid(ffp(concat).squeeze(-1)),dim=-1)

In [399]:
torch.sum(D_T_P[0])

tensor(1.0000, grad_fn=<SumBackward0>)

In [427]:
import ot
def compute_optimal_transport(p, q, C, epsilon=0.05):
    device = p.device

    p_i = p.detach().cpu().numpy()
    q_i = q.detach().cpu().numpy()
    C_i = C.detach().cpu().numpy()

    pi_i = ot.sinkhorn(p_i, q_i, C_i, reg=epsilon)

    pi_i_tensor = torch.tensor(pi_i, device=device)

    return pi_i_tensor

In [428]:
print(D_W_P[0])
print(D_T_P[0])
print(cost_matrix[0])

tensor([0.0921, 0.0961, 0.0943, 0.0922, 0.0912, 0.0915, 0.0877, 0.0861, 0.0822,
        0.0953, 0.0914], grad_fn=<SoftmaxBackward0>)
tensor([0.0918, 0.0882, 0.0929, 0.0894, 0.0941, 0.0892, 0.0920, 0.0894, 0.0907,
        0.0916, 0.0907], grad_fn=<SelectBackward0>)
tensor([[0.2582, 0.2447, 0.2369, 0.2448, 0.2562, 0.2636, 0.2619, 0.2515, 0.2402,
         0.2487, 0.2451],
        [0.2566, 0.2528, 0.2547, 0.2515, 0.2473, 0.2396, 0.2482, 0.2400, 0.2459,
         0.2446, 0.2357],
        [0.2433, 0.2444, 0.2473, 0.2426, 0.2496, 0.2442, 0.2377, 0.2474, 0.2523,
         0.2609, 0.2431],
        [0.2412, 0.2486, 0.2462, 0.2530, 0.2683, 0.2550, 0.2419, 0.2410, 0.2575,
         0.2339, 0.2480],
        [0.2692, 0.2484, 0.2487, 0.2553, 0.2570, 0.2561, 0.2587, 0.2569, 0.2745,
         0.2466, 0.2502],
        [0.2492, 0.2455, 0.2577, 0.2404, 0.2465, 0.2552, 0.2486, 0.2314, 0.2603,
         0.2404, 0.2526],
        [0.2706, 0.2537, 0.2363, 0.2622, 0.2615, 0.2597, 0.2703, 0.2486, 0.2498,
         0.2

In [436]:
pi_i = compute_optimal_transport(D_W_P[0],D_T_P[0],cost_matrix[0])

/Users/nhatminhnguyen/anaconda3/envs/natmin/lib/python3.11/site-packages/ot/bregman/_sinkhorn.py:531: UserWarning: Sinkhorn did not converge. You might want to increase the number of iterations `numItermax` or the regularization parameter `reg`.
  warnings.warn("Sinkhorn did not converge. You might want to "


In [437]:
pi_i.size()

torch.Size([11, 11])

In [439]:
pi_star = []
for sentence in range(len(D_W_P)):
    pi_i = compute_optimal_transport(D_W_P[sentence],D_T_P[sentence],cost_matrix[sentence])
    print(pi_i.size())
    pi_star.append(pi_i)
    

torch.Size([11, 11])
torch.Size([5, 11])
torch.Size([6, 11])
torch.Size([20, 11])


In [440]:
pi_star[0].size()

torch.Size([11, 11])

In [457]:
true_y_3

tensor([1, 2, 1, 0, 2, 1])

In [ ]:
pi_star[2]

tensor([[0.0156, 0.0163, 0.0175, 0.0116, 0.0145, 0.0117, 0.0144, 0.0162, 0.0179,
         0.0147, 0.0167],
        [0.0143, 0.0193, 0.0125, 0.0198, 0.0181, 0.0120, 0.0152, 0.0175, 0.0158,
         0.0135, 0.0192],
        [0.0169, 0.0132, 0.0133, 0.0175, 0.0171, 0.0154, 0.0146, 0.0175, 0.0179,
         0.0150, 0.0152],
        [0.0155, 0.0158, 0.0168, 0.0133, 0.0146, 0.0143, 0.0155, 0.0131, 0.0137,
         0.0170, 0.0110],
        [0.0150, 0.0121, 0.0126, 0.0138, 0.0134, 0.0193, 0.0117, 0.0091, 0.0139,
         0.0174, 0.0168],
        [0.0145, 0.0116, 0.0201, 0.0134, 0.0163, 0.0165, 0.0206, 0.0161, 0.0114,
         0.0140, 0.0118]])

In [450]:
true_y_3 = torch.randint(0,3,[6])

In [454]:
true_y_3.size()

torch.Size([6])

In [453]:
pi_star[2].size()

torch.Size([6, 11])

In [467]:
torch.sum(-torch.log(pi_star[2].gather(1,true_y_3.unsqueeze(1))))

tensor(25.8244)

tensor(0.9933)